In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

1. Caricare in un dataframe il file “bank.csv” (preso da kaggle - https://www.kaggle.com/datasets/mathchi/churn-for-bank-customers) che contiene dati relativi ai clienti di una banca. Lo scopo è predire se il cliente abbandonerà la banca (feature Exited).

In [5]:
df = pd.read_csv('../data/bank.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.shape

(10000, 14)

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


2. Convertire i valori di Gender in numerici (Male = 0, Female = 1).

In [6]:
df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

3. Trasformare la feature Geography con un OneHotEncoder.

In [1]:
from sklearn import preprocessing

In [7]:
enc = preprocessing.OneHotEncoder()
enc.fit(np.array(df['Geography']).reshape(-1,1))

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [8]:
enc.transform(np.array(df['Geography']).reshape(-1,1))

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
pd.DataFrame.sparse.from_spmatrix(enc.transform(np.array(df['Geography']).reshape(-1,1)))

,0,1,2
0,1.0,0,0
1,0,0,1.0
2,1.0,0,0
3,1.0,0,0
4,0,0,1.0
...,...,...,...
9995,1.0,0,0
9996,1.0,0,0
9997,1.0,0,0
9998,0,1.0,0


In [13]:
pd.Series(enc.transform(np.array(df['Geography']).reshape(-1,1)))

KeyError: 'Geography'

In [11]:
df = pd.get_dummies(df, columns=['Geography'], dtype=int)

In [12]:
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,1,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,1,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,1,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,0,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,9997,15569892,Johnstone,516,0,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,9998,15584532,Liu,709,1,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,9999,15682355,Sabbatini,772,0,42,3,75075.31,2,1,0,92888.52,1,0,1,0


4. Dividere il dataset in train (75% del dataset) e test (25%). Usare random_state=0.

In [ ]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Exited', axis=1), df['Exited'], test_size=0.25, random_state=0, stratify=df['Exited'])

5. Allenare un decision tree per predire l’abbandono dei clienti. Settare i parametri a piacere e calcolare l’accuratezza della predizione.

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.796

6. Aggiungere una nuova feature nel dataset con il valore di (EstimatedSalary*Tenure+Balance)/2. L’accuratezza della predizione migliora?

In [ ]:
df['new_feature'] = (df['EstimatedSalary'] * df['Tenure'] + df['Balance']) / 2

X_train, X_test, y_train, y_test = train_test_split(df.drop('Exited', axis=1), df['Exited'], test_size=0.25, random_state=0)
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.7888

7. Applicare due trasformazioni diverse a due feature diverse del dataset per migliorare la predizione.

In [ ]:
import sklearn.preprocessing as preprocessing

balance_norm = pd.DataFrame(preprocessing.normalize(df['Balance'].values.reshape(-1,1), norm='l2'), columns=['NormBalance'])
esalary_norm = pd.DataFrame(preprocessing.normalize(df['EstimatedSalary'].values.reshape(-1,1), norm='l2'), columns=['NormEstimatedSalary'])

In [ ]:
df_norm = pd.concat([df.drop(['Balance', 'EstimatedSalary'], axis=1), balance_norm, esalary_norm], axis=1)

In [ ]:
df_norm.head()

,CreditScore,Gender,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain,new_feature,NormBalance,NormEstimatedSalary
0,619,1,42,2,1,1,1,1,1,0,0,101348.880,0.0,1.0
1,608,1,41,1,1,0,1,0,0,0,1,98175.220,1.0,1.0
2,502,1,42,8,3,1,0,1,1,0,0,535556.680,1.0,1.0
3,699,1,39,1,2,0,0,0,1,0,0,46913.315,0.0,1.0
4,850,1,43,2,1,1,1,0,0,0,1,141839.510,1.0,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_norm.drop('Exited', axis=1), df_norm['Exited'], test_size=0.25, random_state=0)
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.7876

In [ ]:
scaled_age = preprocessing.StandardScaler().fit_transform(df['Age'].values.reshape(-1, 1))

In [ ]:
scaled_age = pd.Series(scaled_age.reshape(-1), name='scaled_age')

In [ ]:
new_df = pd.concat([df.drop('Age', axis=1), scaled_age], axis=1)

In [ ]:
new_df

,CreditScore,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,new_feature,scaled_age
0,619,1,2,0.00,1,1,1,101348.88,1,1,0,0,101348.880,0.293517
1,608,1,1,83807.86,1,0,1,112542.58,0,0,0,1,98175.220,0.198164
2,502,1,8,159660.80,3,1,0,113931.57,1,1,0,0,535556.680,0.293517
3,699,1,1,0.00,2,0,0,93826.63,0,1,0,0,46913.315,0.007457
4,850,1,2,125510.82,1,1,1,79084.10,0,0,0,1,141839.510,0.388871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,5,0.00,2,1,0,96270.64,0,1,0,0,240676.600,0.007457
9996,516,0,10,57369.61,1,1,1,101699.77,0,1,0,0,537183.655,-0.373958
9997,709,1,7,0.00,1,0,1,42085.58,1,1,0,0,147299.530,-0.278604
9998,772,0,3,75075.31,2,1,0,92888.52,1,0,1,0,176870.435,0.293517


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df.drop('Exited', axis=1), new_df['Exited'], test_size=0.25, random_state=0)
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.7972